In [1]:
#Imports
!pip install mysql-connector-python

import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.types import *
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import create_engine
import mysql.connector

import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.6 MB/s eta 0:00:0000:0100:01


**Loading the Data**

In [3]:
basics = pd.read_csv('Data/title_basics.csv.gz')
rating = pd.read_csv('Data/title_ratings.csv.gz')
tmdb = pd.read_csv('Data/tmdb_results_combined.csv.gz')

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [6]:
rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000005,6.2,2622
3,tt0000006,5.1,182
4,tt0000007,5.4,820


In [7]:
tmdb.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
1,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,"{'id': 1131062, 'name': 'Wong Kar-Wai’s Love T...",150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,14204632.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,2247.0,PG
2,tt0119273,0.0,/f5C03doOWiauu37bToKXtpgP5bS.jpg,"{'id': 141086, 'name': 'Heavy Metal Collection...",15000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,16225.0,en,Heavy Metal 2000,...,0.0,88.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,not to survive the fight in the external world...,Heavy Metal 2000,0.0,6.099,186.0,R
3,tt0120263,0.0,/ynXVuylP8upazjz8lrqb1PEMkdR.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,34070.0,sv,Sånger från andra våningen,...,80334.0,99.0,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,NaN,Songs from the Second Floor,0.0,7.200,239.0,NaN
4,tt0120467,0.0,/knok3mNReKqPTplnnqz7E4dd7mD.jpg,NaN,120000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,19085.0,en,Vulgar,...,14904.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone loves a clown... some more than others.,Vulgar,0.0,5.500,44.0,R


In [8]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                639 non-null    object 
 1   adult                  639 non-null    float64
 2   backdrop_path          541 non-null    object 
 3   belongs_to_collection  111 non-null    object 
 4   budget                 639 non-null    float64
 5   genres                 639 non-null    object 
 6   homepage               76 non-null     object 
 7   id                     639 non-null    float64
 8   original_language      639 non-null    object 
 9   original_title         639 non-null    object 
 10  overview               637 non-null    object 
 11  popularity             639 non-null    float64
 12  poster_path            621 non-null    object 
 13  production_companies   639 non-null    object 
 14  production_countries   639 non-null    object 
 15  releas

**Requested Colums/Files**

**title_basics**

Movie ID (tconst)

Primary Title

Start Year

Runtime (in Minutes)

Genre

In [10]:
basics = basics[['tconst', 'primaryTitle', 'startYear', 'endYear', 'runtimeMinutes',
                      'genres']]
basics.head()

,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,Life of a Beijing Policeman,2013.0,NaN,120,"Drama,History"
2,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,NaN,70,Drama
3,tt0069049,The Other Side of the Wind,2018.0,NaN,122,Drama
4,tt0088751,The Naked Monster,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


**tmdb_results**

Movie ID (tconst)

Revenue

Budget

Certification(MPAA Rating)

In [12]:
tmdb = tmdb[['imdb_id', 'budget', 'revenue', 'certification']]
tmdb

,imdb_id,budget,revenue,certification
0,tt0113026,10000000.0,0.0,NaN
1,tt0118694,150000.0,14204632.0,PG
2,tt0119273,15000000.0,0.0,R
3,tt0120263,0.0,80334.0,NaN
4,tt0120467,120000.0,14904.0,R
...,...,...,...,...
634,tt0445841,12500000.0,123021750.0,NaN
635,tt0867181,5000.0,0.0,NaN
636,tt1039952,500000.0,500000.0,NaN
637,tt1764172,250000.0,0.0,NaN
